In [3]:
import os 
import ee
import geemap

In [5]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
Map

Map(center=[29.7662, -95.241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [21]:
# roi = Map.draw_last_feature.geometry()
# print(roi.get(0))

In [ ]:
# dataset_land_cover = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
# Map.addLayer(dataset_land_cover, {}, "NLCD/Land Cover (2016)")
# Map.add_legend(builtin_legend='NLCD', legend_title="Land Cover", layer_name="NLCD/Land Cover (2016)")

In [ ]:
# dataset_land_cover = ee.Image('USGS/NLCD/NLCD2016').select('impervious_descriptor')
# Map.addLayer(dataset_land_cover, {}, "NLCD/Impervious Descriptor (2016)")

# ee_class_table = """
# Value	Color	Description
# 0	00000	Unclassified
# 20	ff0000	Primary road
# 21	ffff00	Secondary road
# 22	0000ff	Tertiary road
# 23	ffffff	Thinned road
# 24	ffc0c5	Non-road non-energy impervious
# 25	eb82eb	Microsoft buildings
# 26	9f1feb	LCMAP impervious
# 27	40dfd0	Wind turbixnes
# 28	79ff00	Well pads
# 29	005f00	Other energy production
# """
# legend_dict = geemap.legend_from_ee(ee_class_table)
# Map.add_legend(legend_title="Impervious Descriptors", legend_dict=legend_dict, layer_name="NLCD/Impervious Descriptor (2016)")


## Land cover vs. existing emissivity map

In [6]:
nlcd_2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
right_layer = geemap.ee_tile_layer(nlcd_2016, {}, 'NLCD 2016')

Map = geemap.Map()
Map.split_map(left_layer='HYBRID', right_layer=right_layer)
Map.add_legend(builtin_legend='NLCD', legend_title="Land Cover", layer_name="NLCD/Land Cover (2016)")
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Landcover: 2016 vs. 2019

In [9]:
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# The collection contains images for multiple years and regions in the USA.
# print('Products:', dataset.aggregate_array('system:index'))

# Filter the collection to the 2016 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Each product has multiple bands for describing aspects of land cover.
# print('Bands:', nlcd2019.bandNames())

# Select the land cover band.
landcover_2019 = nlcd2019.select('landcover')

# Display land cover on the map.
# Map.setCenter(-95, 38, 5)
# Map.addLayer(landcover, None, 'Landcover')


In [10]:
nlcd_2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
left_layer = geemap.ee_tile_layer(nlcd_2016, {}, 'NLCD 2016')
right_layer = geemap.ee_tile_layer(landcover_2019, {}, 'NLCD 2019')
Map = geemap.Map()
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

Left: Landcover from 2016; Right: land cover from 2019

## Landcover vs. LANDSAT 8 RGB

In [7]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
Map

Map(center=[29.7662, -95.241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [8]:
landsat8_2019 = ee.ImageCollection("LANDSAT/LC08/C02/T1_RT_TOA").filterDate('2021-08-01', '2021-09-25')
trueColor432Vis = {
  min: 0.0,
  max: 0.4,}  
trueColor432 = landsat8_2019.select(['B4', 'B3', 'B2'])
Map.addLayer(trueColor432, trueColor432Vis, 'Landsat8-RGB')
# Map

In [12]:
left_layer = geemap.ee_tile_layer(landsat8_2019.select(['B4', 'B3', 'B2']), {min: 0.0, max: 0.4,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(landcover_2019, {}, 'NLCD 2019')
Map = geemap.Map()
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_legend(builtin_legend='NLCD', legend_title="Land Cover", layer_name="NLCD/Land Cover (2016)")
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Emissivity of LANDSAT8 BAND10 estimated from ASTER GED

In [27]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
Map

Map(center=[29.7662, -95.241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [26]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
vis_params_em = {'palette': ['0d0887', 'cc4778', 'f0f921'], 'min': 0.9, 'max': 1.0, 'opacity': 1.0}
houston_emis = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807").select('ST_EMIS').multiply(1/10000.)
Map.add_colorbar(vis_params_em, layer_name="houston_emis", label="Emissivity")
Map.addLayer(houston_emis, vis_params_em, "Emissivity")
Map

Map(center=[29.7662, -95.241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

Problem with **ASTER GED V3**:
* 1 singular **static map** representing the average emissivity from 2000 - 2008
* Lower resolution at 100m
* Algorithm designed and validated for **natural** environments

There is an adjustment mechanism named **ASTER GED V4**, for dynamic annual & inter-annual adjustment upon V3, however:
* Even lower resolution: **5km**
* As time progresses, error of both V3 and V4 increases
* Algorithm is more error prune for **mixed vegetation pixels**, followed by fully vegetated pixels. For desert the total uncertainty was only due to algorithm uncertainty.
* Original author did not discuss this product in urban context. 

Arguments above are summed up from Hulley et al 2015 (2 papers under the same author)

## LANDSAT8 Band 10 Surface Temperature

In [28]:
def applyScaleFactors(image):
  # opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0).add(-273.15)
  return image.addBands(thermalBands, None, True)

Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
houston_temp = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807").select('ST_B10')
houston_temp = applyScaleFactors(houston_temp)
vis_params_temp = { 'palette': ['0d0887', 'cc4778', 'f0f921'], 'min': 20, 'max': 70, 'opacity': 1.0}
Map.add_colorbar(vis_params_temp, label="Surface Temperature (C)", layer_name="houston_temp")
Map.addLayer(houston_temp, vis_params_temp, "Surface Temperature")
Map

Map(center=[29.7662, -95.241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

Problem: 
* **Resolution too low** (~100m), according to ASTERGED guide
* Data products **must contain both optical and thermal data** to be successfully processed to surface temperature, as ASTER NDVI is required to temporally adjust the ASTER GED product to the target Landsat scene. Therefore, night time acquisitions cannot be processed to surface temperature.
* A known **error** exists in the surface temperature retrievals relative to **clouds and possibly cloud shadows**. The characterization of these issues has been documented by Cook et al., (2014).


In [29]:
Map = geemap.Map(center=(29.7163243, -95.4109632,), zoom=15) # coordinates for Rice University
Map.split_map(left_layer='HYBRID', right_layer=geemap.ee_tile_layer(houston_temp, vis_params_temp, "Surface Temperature"))
Map

Map(center=[29.7163243, -95.4109632], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

## Is one tile sufficient across cycles?

In [17]:
Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)
cycle1 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807").select('SR_B1')
cycle2 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210823").select('SR_B1')
cycle3 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210908").select('SR_B1')
cycle4 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210924").select('SR_B1')
Map.addLayer(cycle1)
Map.addLayer(cycle2)
Map.addLayer(cycle3)
Map.addLayer(cycle4)
Map

Map(center=[29.7662, -95.241], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

Yes. The boundary of each tile only drifts by ~ pixels, if at all.


## Is every band of LANDSAT 8 affected by cloud?

In [22]:
def scaled(image):  # for LANDSAT8 surfance reflectance
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

In [23]:
BOUNDING_BOX = [[[-95.690165, 29.5937], [-95.690165, 30.266005], [-94.900379, 30.266005], [-94.900379, 29.5937]]]

Map = geemap.Map(center=(29.7662, -95.2410), zoom=9)

sample_tile_scaled = scaled(ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_025039_20210807"))

vis_params = [
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B1'], 'min': 0, 'max': 0.3},
    {'bands': ['SR_B5'], 'min': 0, 'max': 0.4},
    {'bands': ['SR_B6'], 'min': 0, 'max': 0.3},
    {'bands': ['ST_B10'], 'min': 300, 'max': 500},
    {'bands': ['SR_B7'], 'min': 0, 'max': 0.3},
]

labels = [
    'Natural Color (SR_B4/SR_B3/SR_B2)',
    'Ultra blue (SR_B1)',
    'NIR (SR_B5)',
    'SWIR1 (SR_B6)',
    'TIR (ST_B10)',
    'SWIR2 (SR_B7)',
]

geemap.linked_maps(
    rows=2,
    cols=3,
    height="400px",
    center=[29.7662, -95.2410],
    zoom=10,
    ee_objects=[sample_tile_scaled],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)



GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

![title](./fig/Landsat.v.Sentinel-2.png)

In [24]:
left_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['SR_B4', 'SR_B3', 'SR_B2']), {'min': 0.0, 'max': 0.3,}, 'Landsat8')
right_layer = geemap.ee_tile_layer(sample_tile_scaled.select(['ST_B10']), {'min': 300, 'max': 350,}, 'TIR')
Map = geemap.Map(center=(29.7662, -95.2410), zoom=14)
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.add_colorbar({'min': 320, 'max': 370,}, label="Brightness Temperature (K)")
Map

Map(center=[29.7662, -95.241], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

For thermal band, the decrease in pixel value is actually caused by the cloud itself, not its shadow

## Pre-processing Emissivity

### Original Legend
**11	Open water**: areas of open water, generally with less than 25% cover of vegetation or soil.</br>
**12	Perennial ice/snow:** areas characterized by a perennial cover of ice and/or snow, generally greater than 25% of total cover.</br>
**21	Developed, open space:** areas with a mixture of some constructed materials, but mostly vegetation in the form of lawn grasses. Impervious surfaces account for less than 20% of total cover. These areas most commonly include large-lot single-family housing units, parks, golf courses, and vegetation planted in developed settings for recreation, erosion control, or aesthetic purposes.</br>
**22	Developed, low intensity:** areas with a mixture of constructed materials and vegetation. Impervious surfaces account for 20% to 49% percent of total cover. These areas most commonly include single-family housing units.</br>
**23	Developed, medium intensity:** areas with a mixture of constructed materials and vegetation. Impervious surfaces account for 50% to 79% of the total cover. These areas most commonly include single-family housing units.</br>
**24	Developed high intensity:** highly developed areas where people reside or work in high numbers. Examples include apartment complexes, row houses, and commercial/industrial. Impervious surfaces account for 80% to 100% of the total cover.</br>
**31	Barren land (rock/sand/clay):** areas of bedrock, desert pavement, scarps, talus, slides, volcanic material, glacial debris, sand dunes, strip mines, gravel pits, and other accumulations of earthen material. Generally, vegetation accounts for less than 15% of total cover.</br>
**41	Deciduous forest:** areas dominated by trees generally greater than 5 meters tall, and greater than 20% of total vegetation cover. More than 75% of the tree species shed foliage simultaneously in response to seasonal change.</br>
**42	Evergreen forest:** areas dominated by trees generally greater than 5 meters tall, and greater than 20% of total vegetation cover. More than 75% of the tree species maintain their leaves all year. Canopy is never without green foliage.</br>
**43	Mixed forest:** areas dominated by trees generally greater than 5 meters tall, and greater than 20% of total vegetation cover. Neither deciduous nor evergreen species are greater than 75% of total tree cover.</br>
**51	Dwarf scrub:** *Alaska only* areas dominated by shrubs less than 20 centimeters tall with shrub canopy typically greater than 20% of total vegetation. This type is often co-associated with grasses, sedges, herbs, and non-vascular vegetation.</br>
**52	Shrub/scrub:** areas dominated by shrubs less than 5 meters tall with shrub canopy typically greater than 20% of total vegetation. This class includes true shrubs, young trees in an early successional stage, or trees stunted from environmental conditions.</br>
**71	Grassland/herbaceous:** areas dominated by gramanoid or herbaceous vegetation, generally greater than 80% of total vegetation. These areas are not subject to intensive management such as tilling, but can be utilized for grazing.</br>
**72	Sedge/herbaceous:** *Alaska only* areas dominated by sedges and forbs, generally greater than 80% of total vegetation. This type can occur with significant other grasses or other grass like plants, and includes sedge tundra and sedge tussock tundra.</br>
**73	Lichens:** *Alaska only* areas dominated by fruticose or foliose lichens generally greater than 80% of total vegetation.</br>
**74	Moss:** *Alaska only areas* dominated by mosses, generally greater than 80% of total vegetation.</br>
**81	Pasture/hay:** areas of grasses, legumes, or grass-legume mixtures planted for livestock grazing or the production of seed or hay crops, typically on a perennial cycle. Pasture/hay vegetation accounts for greater than 20% of total vegetation.</br>
**82	Cultivated crops:** areas used for the production of annual crops, such as corn, soybeans, vegetables, tobacco, and cotton, and also perennial woody crops such as orchards and vineyards. Crop vegetation accounts for greater than 20% of total vegetation. This class also includes all land being actively tilled.</br>
**90	Woody wetlands:** areas where forest or shrubland vegetation accounts for greater than 20% of vegetative cover and the soil or substrate is periodically saturated with or covered with water.</br>
**95	Emergent herbaceous wetlands:** areas where perennial herbaceous vegetation accounts for greater than 80% of vegetative cover and the soil or substrate is periodically saturated with or covered with water.</br>

### Observations
* There are 20 classes in total 
* Some classes are Alkakan only (down to 14)
* those who share the same first-digit are in the same parent class
* Mixture percentage between two classes (e.g. developed open space 20% vs. Developed high intensity > 80%)lends itself to linear interpolation

## Notekeeping

Dimension of LANDSAT8 Houston tile: (7741, 7871)